In [ ]:
## download necessary files
!dx download ./results/ukb_demographics_matched.csv
!dx download ./results/fs_derivatives.html

!dx download ./results/ukb_vol.csv
!dx download ./results/ukb_thickness.csv
!dx download ./results/ukb_area.csv

now run harmonization

In [ ]:
!pip install neuroHarmonize
!pip install neuroCombat
!pip install statsmodels
!pip install nibabel
!pip install lxml
!pip install --upgrade seaborn
# !pip install html5lib

In [ ]:
import pandas as pd
from neuroHarmonize import harmonizationLearn, harmonizationApply

In [ ]:
## read the dataframes
df_covar = pd.read_csv("ukb_demographics_matched.csv")
df_features = pd.read_csv("ukb_vol.csv")
df_map = pd.read_html("fs_derivatives.html")[1]

## organize df_map
df_map = df_map.iloc[1:].reset_index(drop=True)
df_map["UDI"] = (
                    "p"
                    + df_map["UDI"].astype(str)
                    .str.replace("-", "_i", regex=False)
                    .str.replace(r"\.0$", "", regex=True)
                )
df_map = df_map[["UDI", "Description"]]

## keep necessary columns from features
feature_cols = df_map["UDI"].tolist()
vol_features = ["eid"] + [f for f in feature_cols if f in df_features.columns]
df_features = df_features[vol_features]

## clean df_covars
srt_cols = [
            "speechreceptionthreshold_srt_estimate_left",
            "speechreceptionthreshold_srt_estimate_right"
            ]
df_covar["srt"] = df_covar[srt_cols].sum(axis=1)
df_covar.drop(columns=srt_cols, inplace=True)

covar_cols = ["subject_id",	"sex", "age", "center_v2", "center_v3", "tin_status", "tin_severity", "tin_duration", "srt"]
df_covar = df_covar[covar_cols]
df_covar["SITE"] = df_covar["center_v2"].combine_first(df_covar["center_v3"])
df_covar.drop(columns=["center_v2", "center_v3"], inplace=True)

## merge covar with features
df_features.rename(columns={"eid" : "subject_id"}, inplace=True)
df_covar = df_covar.merge(df_features, on="subject_id", how="left")
i2_cols = [c for c in df_covar.columns if c.endswith("_i2")]

for c in i2_cols:
    base = c[:-3]
    c_i2 = c
    c_i3 = base + "_i3"

    df_covar[base] = df_covar[c_i2].combine_first(df_covar[c_i3])

df = df_covar.drop(columns=[c for c in df_covar.columns if c.endswith(("_i2", "_i3"))])

In [ ]:
## drop some nans in data (happened in fields p27531 p26588 and p26589 for 3 subjects)
cols_to_check = df.columns.difference(["tin_severity", "tin_duration"])
df = df.dropna(subset=cols_to_check, how="any")

## get controls and create matrixes
df_subjects = df[["subject_id", "tin_status", "tin_severity", "tin_duration"]]
df_covars = df[["age", "sex", "srt", "SITE"]]
feature_cols = [c for c in df.columns if c.startswith("p")]
data_matrix = df[feature_cols].to_numpy()

df_controls = df.query('tin_status == 0')
controls_matrix = df_controls.filter(regex="^p").to_numpy()
controls_covars = df_controls[["age", "sex", "srt", "SITE"]]

for col in ['sex', 'SITE']:
    df_covars[col] = df_covars[col].astype(str)
    controls_covars[col] = controls_covars[col].astype(str)

## run harmonizing and get model and apply on all
hm_model, _ = harmonizationLearn(controls_matrix, controls_covars)
my_data_adj = harmonizationApply(data_matrix, df_covars, hm_model)

## return back to dataframe
df_hm = pd.concat([
                        df_subjects.reset_index(drop=True),
                        df_covars.reset_index(drop=True),
                        pd.DataFrame(my_data_adj, columns=feature_cols)
                        ],
                        axis=1
                        )
df_hm.to_csv("ukb_vol_harmonized.csv", index=False)
!dx upload ukb_vol_harmonized.csv --dest / 

Statistical comparison with different atlases

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests

In [ ]:
## read harmonized df
df_hm = pd.read_csv('ukb_vol_harmonized.csv')
atlas_name = "hippo_subfields"
covariates = ["age", "sex", "srt"]  
correction = "bonferroni"
alpha = 0.05

In [ ]:
## ranges from UKB documentation
FS_RANGES = {
    "aparc_volume": {
        "lh": (27205, 27235),
        "rh": (27298, 27328),
    },
    "aparc_2009_volume": {
        "lh": (27477, 27550),
        "rh": (27699, 27772),
    },
    "amygdalar_nuclei": {
        "lh": (26600, 26609),
        "rh": (26610, 26619),
    },
    "brainstem": {
        "both": (26716, 26720),
    },
    "hippo_subfields": {
        "lh": (26620, 26641),
        "rh": (26642, 26663),
    },
    "aseg": {
        "lh": (26554, 26567),
        "rh": (26585, 26598),
    },
    "thalamic_nuclei": {
        "lh": (26664, 26687),
        "rh": (26688, 26715),
    },
    "aparc_thickness": {
        "lh": (26756, 26788),
        "rh": (26857, 26889),
    },
    "aparc_area": {
        "lh": (26722, 26754),
        "rh": (26823, 26855),
    },
    "aparc_2009s_thickness": {
        "lh": (27403, 27476),
        "rh": (27625, 27698),
    },
    "aparc_2009s_area": {
        "lh": (27329, 27402),
        "rh": (27625, 27698),
    },
}

ATLAS_RULES = {
    "thalamic_nuclei": {
        "sum": {
            "Volume_of_Whole_thalamus": [
                "Volume_of_Whole_thalamus_left_hemisphere",
                "Volume_of_Whole_thalamus_right_hemisphere",
            ]
        }
    },
    "amygdalar_nuclei": {
        "sum": {
            "Volume_of_Whole_amygdala": [
                "Volume_of_Whole_amygdala_left_hemisphere",
                "Volume_of_Whole_amygdala_right_hemisphere",
            ]
        }
    },
    "hippo_subfields": {
        "sum": {
            "Volume_of_Whole_hippocampus": [
                "Volume_of_Whole_hippocampus_left_hemisphere",
                "Volume_of_Whole_hippocampus_right_hemisphere",
            ]
        },
        "drop": [
            "Volume_of_Whole_hippocampal_body_left_hemisphere",
            "Volume_of_Whole_hippocampal_head_left_hemisphere",
            "Volume_of_Hippocampal_tail_left_hemisphere",
            "Volume_of_Whole_hippocampal_body_right_hemisphere",
            "Volume_of_Whole_hippocampal_head_right_hemisphere",
            "Volume_of_Hippocampal_tail_right_hemisphere",
        ],
    },
}

In [ ]:
## define a function to select necessary columns
def pcols_from_ranges(ranges):
    cols = []
    for r in ranges.values():
        cols.extend(range(r[0], r[1] + 1))
    return [f"p{i}" for i in cols]

common_cols = list(df_hm.columns[:7])

## get the atlas
sel_cols = pcols_from_ranges(FS_RANGES[atlas_name])
df_atlas = df_hm[
                    common_cols + sel_cols
                        ]

## map names in the UKB mapping to atlas columns
# df_map = df_map.iloc[1:].reset_index(drop=True)
df_map_1 = df_map.copy()
df_map_1["UDI"] = (
                    df_map_1["UDI"].astype(str)
                    .str.replace("-", "_i", regex=False)
                    .str.replace(r"\.0$", "", regex=True)
                )
df_map_1 = df_map_1[["UDI", "Description"]]
df_map_1 = df_map_1[df_map_1["UDI"].str.endswith("_i2", na=False)].copy()
df_map_1["UDI"] = df_map_1["UDI"].str.replace("_i2$", "", regex=True)
rename_map = dict(zip(df_map_1["UDI"], df_map_1["Description"]))
df_atlas.rename(columns=rename_map, inplace=True)
df_atlas.columns = (
                    df_atlas.columns
                    .str.replace(r"[ \-\+]", "_", regex=True)
                    .str.replace(r"[()]", "", regex=True)
                    )

## some name adjustments for special atlases
rules = ATLAS_RULES.get(atlas_name, {})

for new_col, cols in rules.get("sum", {}).items():
    df_atlas[new_col] = df_atlas[cols].sum(axis=1)
    df_atlas.drop(columns=cols, inplace=True)

if "drop" in rules:
    df_atlas.drop(columns=rules["drop"], inplace=True)

end = None if atlas_name == "aparc_2009_volume" else -1
bl_cols = list(df_atlas.columns[7:end])

In [ ]:
## function to run ANOVA + multiple comparison
def mass_ancova(
    df,
    outcome_cols,
    group_col,
    covariates,
    correction="fdr_bh",
    alpha=0.05,
):
    results = []

    for col in outcome_cols:
        formula = f"{col} ~ {group_col} + " + " + ".join(covariates)

        try:
            model = smf.ols(formula, data=df).fit()

            beta = model.params[group_col]
            pval = model.pvalues[group_col]
            tval = model.tvalues[group_col]

            n1 = (df[group_col] == 1).sum()
            n0 = (df[group_col] == 0).sum()

            mean_tinnitus = df.loc[df[group_col] == 1, col].mean()
            mean_control = df.loc[df[group_col] == 0, col].mean()

            cohen_d = tval * np.sqrt(1 / n1 + 1 / n0)

            results.append({
                "brain_label": col,
                "beta": beta,
                "t": tval,
                "pval": pval,
                "cohen_d": cohen_d,
                "n_tinnitus": n1,
                "n_control": n0,
                "mean_tinnitus": mean_tinnitus,
                "mean_control": mean_control,
            })

        except Exception as e:
            results.append({
                "brain_label": col,
                "beta": np.nan,
                "t": np.nan,
                "pval": np.nan,
                "cohen_d": np.nan,
                "n_tinnitus": np.nan,
                "n_control": np.nan,
                "mean_tinnitus": np.nan,
                "mean_control": np.nan,
                "error": str(e),
            })

    res = pd.DataFrame(results)

    # multiple testing correction
    mask = res["pval"].notna()

    _, pval_adj, _, _ = multipletests(
        res.loc[mask, "pval"],
        alpha=alpha,
        method=correction,
    )

    res.loc[mask, "pval_adj"] = pval_adj
    res["significant"] = res["pval_adj"] < alpha

    return res

df_results = mass_ancova(
                        df_atlas,
                        bl_cols,
                        group_col="tin_status",
                        covariates=covariates,
                        correction=correction,
                        alpha=0.05,
                    )
df_results.sort_values(by="pval", inplace=True)
df_sig = df_results.query('significant == True')
df_atlas["tin_status"] = df_atlas["tin_status"].map({1: "Tinnitus", 0: "Control"})

Plotting significant results

In [ ]:
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import pearsonr

In [ ]:
def plot_sig_rois(df, roi):

    pal = ['#1f77b4', '#d62728']
    order = ["Control", "Tinnitus"]

    fig, ax = plt.subplots(1, 1, figsize=(6, 2.3), layout="tight")

    sns.boxplot(
        data=df,
        y="tin_status",
        x=roi,
        palette=pal,
        width=0.5,
        linewidth=1.8,
        fill=False,
        order=order,
        showfliers=False,
        ax=ax
    )

    sns.stripplot(
        data=df,
        y="tin_status",
        x=roi,
        palette=pal,
        linewidth=0,
        size=3.5,
        edgecolor=None,
        jitter=0.25,
        alpha=0.1,
        order=order,
        ax=ax
    )

    # Clean axes
    ax.spines[['top', 'left', 'right']].set_visible(False)
    ax.set(ylabel="", xlabel="", yticks=[], title=roi)

    handles = [
        Patch(facecolor='none', edgecolor=pal[i], linewidth=2, label=order[i])
        for i in range(len(pal))
    ]
    ax.legend(
        handles=handles,
        title="Group",
        loc='upper left',
        bbox_to_anchor=(1.001, 0.99),
        borderaxespad=0,
        frameon=False
    )

rois = df_sig['brain_label'].values.tolist()
for roi in rois:
    plot_sig_rois(df_atlas, roi)

Plotting significant correlations (severity + duration)

In [ ]:
mode = "tin_severity"
covariates = ("age", "sex", "srt")
correction = "fdr_bh"

In [ ]:
def mass_partial_corr(
    df,
    mode,
    feature_cols=None,
    covariates=("age", "sex", "srt"),
    correction="fdr_bh",
    alpha=0.05,
):
    results = []

    X = sm.add_constant(df[list(covariates)])
    y = df[mode]
    sev_resid = sm.OLS(y, X, missing="drop").fit().resid

    for col in feature_cols:
        tmp = df[[col] + list(covariates)].dropna()

        if tmp.shape[0] < 10:
            results.append({
                "feature": col,
                "r": np.nan,
                "pval": np.nan,
                "n": tmp.shape[0],
            })
            continue

        X_tmp = sm.add_constant(tmp[list(covariates)])
        feat_resid = sm.OLS(tmp[col], X_tmp).fit().resid

        r, p = pearsonr(sev_resid.loc[tmp.index], feat_resid)

        results.append({
            "feature": col,
            "r": r,
            "pval": p,
            "n": tmp.shape[0],
        })

    res = pd.DataFrame(results)

    # multiple comparison correction
    mask = res["pval"].notna()
    _, p_adj, _, _ = multipletests(
        res.loc[mask, "pval"],
        method=correction,
        alpha=alpha
    )

    res.loc[mask, "pval_adj"] = p_adj
    res["significant"] = res["pval_adj"] < alpha

    return res.sort_values("pval_adj")


df_corr = df_atlas.query('tin_status == "Tinnitus"')
df_tis = df_corr.dropna(subset=["tin_severity"], how="any")
df_tid = df_corr.dropna(subset=["tin_duration"], how="any")

mode = "tin_duration"
if mode == "tin_severity":
    df = df_tis.copy()
elif mode == "tin_duration":
    df = df_tid.copy()
else:
    raise ValueError("wrong mode.")

feature_cols = list(df.columns[7:])
df_corr_results = mass_partial_corr(
    df,
    mode=mode,
    feature_cols=feature_cols,
    covariates=covariates,
    correction=correction
)
df_corr_results = df_corr_results.query('significant == True')
df_corr_results

Check deviation scores